In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [144]:
headers = {'User-Agent': 'Mozilla/5.0'}
url = 'https://en.wikipedia.org/wiki/List_of_highest-grossing_films'
response = requests.get(url, headers=headers)
if response.status_code == 200:
    page = BeautifulSoup(response.content, "html")

In [145]:
table = page.find('table', {'class': 'wikitable sortable plainrowheaders sticky-header col4right col5center col6center'})

In [146]:
films = []
film_links = {}

films_idx = 0
for tr in table.select('tr')[1:]:
    td = tr.select('td')
    title = tr.select_one('th')
    data = {
        'title': title.text[:-1],
        'year': td[3].text[:-1],
        'director': '',
        'box_office': 0,
        'box_office_currency': '',
        'country': ''
    }
    
    link = title.select_one('a')['href']
    film_links[films_idx] = link
    
    films.append(data)
    films_idx += 1

In [147]:
films

[{'title': 'Avatar',
  'year': '2009',
  'director': '',
  'box_office': 0,
  'box_office_currency': '',
  'country': ''},
 {'title': 'Avengers: Endgame',
  'year': '2019',
  'director': '',
  'box_office': 0,
  'box_office_currency': '',
  'country': ''},
 {'title': 'Avatar: The Way of Water',
  'year': '2022',
  'director': '',
  'box_office': 0,
  'box_office_currency': '',
  'country': ''},
 {'title': 'Titanic',
  'year': '1997',
  'director': '',
  'box_office': 0,
  'box_office_currency': '',
  'country': ''},
 {'title': 'Star Wars: The Force Awakens',
  'year': '2015',
  'director': '',
  'box_office': 0,
  'box_office_currency': '',
  'country': ''},
 {'title': 'Avengers: Infinity War',
  'year': '2018',
  'director': '',
  'box_office': 0,
  'box_office_currency': '',
  'country': ''},
 {'title': 'Spider-Man: No Way Home',
  'year': '2021',
  'director': '',
  'box_office': 0,
  'box_office_currency': '',
  'country': ''},
 {'title': 'Inside Out 2',
  'year': '2024',
  'direct

In [148]:
film_links

{0: '/wiki/Avatar_(2009_film)',
 1: '/wiki/Avengers:_Endgame',
 2: '/wiki/Avatar:_The_Way_of_Water',
 3: '/wiki/Titanic_(1997_film)',
 4: '/wiki/Star_Wars:_The_Force_Awakens',
 5: '/wiki/Avengers:_Infinity_War',
 6: '/wiki/Spider-Man:_No_Way_Home',
 7: '/wiki/Inside_Out_2',
 8: '/wiki/Jurassic_World',
 9: '/wiki/Ne_Zha_2',
 10: '/wiki/The_Lion_King_(2019_film)',
 11: '/wiki/The_Avengers_(2012_film)',
 12: '/wiki/Furious_7',
 13: '/wiki/Top_Gun:_Maverick',
 14: '/wiki/Frozen_2',
 15: '/wiki/Barbie_(film)',
 16: '/wiki/Avengers:_Age_of_Ultron',
 17: '/wiki/The_Super_Mario_Bros._Movie',
 18: '/wiki/Black_Panther_(film)',
 19: '/wiki/Harry_Potter_and_the_Deathly_Hallows_%E2%80%93_Part_2',
 20: '/wiki/Deadpool_%26_Wolverine',
 21: '/wiki/Star_Wars:_The_Last_Jedi',
 22: '/wiki/Jurassic_World:_Fallen_Kingdom',
 23: '/wiki/Frozen_(2013_film)',
 24: '/wiki/Beauty_and_the_Beast_(2017_film)',
 25: '/wiki/Incredibles_2',
 26: '/wiki/The_Fate_of_the_Furious',
 27: '/wiki/Iron_Man_3',
 28: '/wiki/Mi

In [175]:
def extract_from_film_page(link, data) -> bool:
    film_page_resp = requests.get(f'https://en.wikipedia.org{link}', headers=headers)
    if film_page_resp.status_code == 200:
        film_page = BeautifulSoup(film_page_resp.content, 'html')
    else:
        return False
    
    table = film_page.find('table', {'class': 'infobox vevent'})
    for tr in table.select('tr'):
        if tr.select_one('th') == None:
            continue
        th = tr.select_one('th').get_text().strip()
        if th == 'Directed by':
            data['director'] = tr.select_one('td').text.strip()
        elif 'Countr' in th:
            data['country'] = tr.select_one('td').text.strip()  
        elif 'office' in th:
            row = tr.find('td').text.split()
            data['box_office'] = row[0]
            data['box_office_currency'] = row[1]
    return True

In [176]:
for idx, link in film_links.items():
    if extract_from_film_page(link, films[idx]) == False:
        print('Error while parsing in:', idx, link)

In [177]:
films

[{'title': 'Avatar',
  'year': '2009',
  'director': 'James Cameron',
  'box_office': '$2.923',
  'box_office_currency': 'billion[5]',
  'country': 'United Kingdom[2]\nUnited States[2]'},
 {'title': 'Avengers: Endgame',
  'year': '2019',
  'director': 'Anthony RussoJoe Russo',
  'box_office': '$2.799',
  'box_office_currency': 'billion[4]',
  'country': 'United States'},
 {'title': 'Avatar: The Way of Water',
  'year': '2022',
  'director': 'James Cameron',
  'box_office': '$2.320',
  'box_office_currency': 'billion[4][5]',
  'country': 'United States'},
 {'title': 'Titanic',
  'year': '1997',
  'director': 'James Cameron',
  'box_office': '$2.264',
  'box_office_currency': 'billion[7]',
  'country': 'United States'},
 {'title': 'Star Wars: The Force Awakens',
  'year': '2015',
  'director': 'J. J. Abrams',
  'box_office': '$2.07',
  'box_office_currency': 'billion[3]',
  'country': 'United States'},
 {'title': 'Avengers: Infinity War',
  'year': '2018',
  'director': 'Anthony RussoJoe

In [193]:
df = pd.DataFrame(films)
df.head()

,title,year,director,box_office,box_office_currency,country
0,Avatar,2009,James Cameron,$2.923,billion[5],United Kingdom[2]\nUnited States[2]
1,Avengers: Endgame,2019,Anthony RussoJoe Russo,$2.799,billion[4],United States
2,Avatar: The Way of Water,2022,James Cameron,$2.320,billion[4][5],United States
3,Titanic,1997,James Cameron,$2.264,billion[7],United States
4,Star Wars: The Force Awakens,2015,J. J. Abrams,$2.07,billion[3],United States


In [195]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   title                50 non-null     object
 1   year                 50 non-null     object
 2   director             50 non-null     object
 3   box_office           50 non-null     object
 4   box_office_currency  50 non-null     object
 5   country              50 non-null     object
dtypes: object(6)
memory usage: 2.5+ KB


In [179]:
import re

In [221]:
for i, row in df.iterrows():
    for key, cell in row.to_dict().items():
        change = re.findall('\[.*.\]', cell)
        cell_ = cell
        for c in change:
            cell_ = cell_.replace(c, '')
        df.loc[df.index == i, key] = cell_


In [227]:
for i, text in enumerate(df['box_office']):
    t = text[text.find('$')+1:]
    df.loc[df.index == i, 'box_office'] = t

In [229]:
df['year'] = pd.to_numeric(df['year'], downcast='integer')
df['box_office'] = pd.to_numeric(df['box_office'], downcast='float')

In [230]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                50 non-null     object 
 1   year                 50 non-null     int16  
 2   director             50 non-null     object 
 3   box_office           50 non-null     float32
 4   box_office_currency  50 non-null     object 
 5   country              50 non-null     object 
dtypes: float32(1), int16(1), object(4)
memory usage: 2.0+ KB


In [12]:
df = df.drop(['box_office_currency'], axis=1)

In [14]:
df.to_csv('films.csv', index=False)

In [15]:
df

,title,year,director,box_office,country
0,Avatar,2009,James Cameron,2.923,United Kingdom\nUnited States
1,Avengers: Endgame,2019,Anthony RussoJoe Russo,2.799,United States
2,Avatar: The Way of Water,2022,James Cameron,2.320,United States
3,Titanic,1997,James Cameron,2.264,United States
4,Star Wars: The Force Awakens,2015,J. J. Abrams,2.070,United States
5,Avengers: Infinity War,2018,Anthony RussoJoe Russo,2.052,United States
6,Spider-Man: No Way Home,2021,Jon Watts,1.953,United States
7,Inside Out 2,2024,Kelsey Mann,1.699,United States
8,Jurassic World,2015,Colin Trevorrow,1.671,United States
9,Ne Zha 2 †,2025,Jiaozi,1.660,China


In [2]:
df = pd.read_csv('films.csv')

In [3]:
df.head()

,title,year,director,box_office,country
0,Avatar,2009,James Cameron,2.923,United Kingdom\nUnited States
1,Avengers: Endgame,2019,Anthony RussoJoe Russo,2.799,United States
2,Avatar: The Way of Water,2022,James Cameron,2.320,United States
3,Titanic,1997,James Cameron,2.264,United States
4,Star Wars: The Force Awakens,2015,J. J. Abrams,2.070,United States


In [9]:
df = df.replace('\n', ', ', regex=True)

In [20]:
df = df.replace('United States China', 'United States, China', regex=True)

In [21]:
df.to_csv('films.csv', index=False)

In [22]:
import sqlite3

In [23]:
connection = sqlite3.connect('films.db')
cursor = connection.cursor()

In [26]:
cursor.execute('DROP TABLE films')

In [27]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS films (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        release_year INTEGER,
        director TEXT,
        box_office REAL,
        country TEXT
    )
''')

In [28]:
req = 'INSERT INTO films (title, release_year, director, box_office, country) VALUES (?, ?, ?, ?, ?)'
for i, row in df.iterrows():
    cursor.execute(req, row.tolist())

In [29]:
cursor.execute('''
SELECT * FROM films 
''')
cursor.fetchall()

[(1, 'Avatar', 2009, 'James Cameron', 2.923, 'United Kingdom, United States'),
 (2,
  'Avengers: Endgame',
  2019,
  'Anthony RussoJoe Russo',
  2.799,
  'United States'),
 (3, 'Avatar: The Way of Water', 2022, 'James Cameron', 2.32, 'United States'),
 (4, 'Titanic', 1997, 'James Cameron', 2.264, 'United States'),
 (5,
  'Star Wars: The Force Awakens',
  2015,
  'J. J. Abrams',
  2.07,
  'United States'),
 (6,
  'Avengers: Infinity War',
  2018,
  'Anthony RussoJoe Russo',
  2.052,
  'United States'),
 (7, 'Spider-Man: No Way Home', 2021, 'Jon Watts', 1.953, 'United States'),
 (8, 'Inside Out 2', 2024, 'Kelsey Mann', 1.699, 'United States'),
 (9, 'Jurassic World', 2015, 'Colin Trevorrow', 1.671, 'United States'),
 (10, 'Ne Zha 2 †', 2025, 'Jiaozi', 1.66, 'China'),
 (11, 'The Lion King', 2019, 'Jon Favreau', 1.657, 'United States'),
 (12, 'The Avengers', 2012, 'Joss Whedon', 1.521, 'United States'),
 (13, 'Furious 7', 2015, 'James Wan', 1.515, 'United States, China'),
 (14, 'Top Gun: Ma

In [30]:
# save updates and close db connection
connection.commit()
connection.close()

In [31]:
!sqlite3 films.db '.mode json' '.once films_ready.json' 'SELECT * FROM films'